## This notebook demonstrates how to perform bulk inference of [Phi-3-mini-128k-instruct](https://huggingface.co/microsoft/Phi-3-mini-128k-instruct) on the [Tracto.ai](https://tracto.ai/) platform.

Tracto is perfect for offline batch inference:

* Easy scaling - just change the `job_count` parameter.
* Fault tolerance out of the box - if a job crashes, it gets restarted automatically. No need to handle GPU failures or infrastructure issues - Tracto takes care of it.
* Full Tracto integration - save results directly to the distributed file system and process them further on the platform.

In [1]:
import yt.wrapper as yt
import uuid

username = yt.get_user_name()
if yt.exists(f"//sys/users/{username}/@user_info/home_path"):
    home = yt.get(f"//sys/users/{username}/@user_info/home_path")
    working_dir = f"{home}/tmp/{uuid.uuid4().hex}"
else:
    working_dir = f"//tmp/examples/{uuid.uuid4().hex}"
yt.create("map_node", working_dir, ignore_existing=True)
print(working_dir)

//home/zippy_salmon_hyena/tmp/b4484db1f8794585b3148b05b212c58d


Prepare data for inference as a table.

In [3]:
from datasets import load_dataset

dataset = load_dataset("Rapidata/Other-Animals-10")

table_path = f"{working_dir}/questions"
yt.create("table", table_path, force=True)

questions = [
    {"question": f"Can a {animal} fly? If it can't - how can we help it take off?"}
    for animal in set(dataset["train"].features["label"].int2str(dataset["train"]["label"]))
]

yt.write_table(table_path, questions)

Run bulk inference of `Phi-3-mini`.

In [5]:
from typing import Iterable
import logging
import sys
import random

yt.config["pickling"]["safe_stream_mode"] = False


result_path = f"{working_dir}/result"


@yt.aggregator
def bulk_inference(records: Iterable[dict[str, str]]) -> dict[str, str]:
    from vllm import LLM, SamplingParams

    llm = LLM(model="microsoft/Phi-3-mini-128k-instruct", trust_remote_code=True)
    sampling_params = SamplingParams(
        temperature=0.6,
        top_p=0.9,
        max_tokens=32000,
    )

    conversations = [
        [
            {
                "role": "user",
                "content": record["question"],
            },
        ]
        for record in records
    ]
    outputs = llm.chat(
        messages=conversations,
        sampling_params=sampling_params,
    )
    for prompt, output in zip(conversations, outputs):
        yield {
            "prompt": prompt,
            "text": output.outputs[0].text,
        }


yt.run_map(
    bulk_inference,
    table_path,
    result_path,
    job_count=2,
    spec={
        "pool_trees": ["gpu_h100"],
        "mapper": {
            "gpu_limit": 1,
            "memory_limit": 32212254720,
            "cpu_limit": 20,
        },
    },
)

2025-05-26 17:07:17,823	INFO	Operation started: https://playground.tracto.ai/playground/operations/99c8304f-c43f5bf8-24dd03e8-4e9bdd9f/details


2025-05-26 17:07:17,868	INFO	( 0 min) operation 99c8304f-c43f5bf8-24dd03e8-4e9bdd9f starting


2025-05-26 17:07:18,410	INFO	( 0 min) operation 99c8304f-c43f5bf8-24dd03e8-4e9bdd9f initializing


2025-05-26 17:07:19,011	INFO	( 0 min) Unrecognized spec: {'enable_partitioned_data_balancing': false, 'mapper': {'title': 'bulk_inference'}}


2025-05-26 17:07:19,065	INFO	( 0 min) operation 99c8304f-c43f5bf8-24dd03e8-4e9bdd9f: running=0     completed=0     pending=2     failed=0     aborted=0     lost=0     total=2     blocked=0    


2025-05-26 17:07:21,993	INFO	( 0 min) operation 99c8304f-c43f5bf8-24dd03e8-4e9bdd9f: running=1     completed=0     pending=1     failed=0     aborted=0     lost=0     total=2     blocked=0    


2025-05-26 17:08:58,543	INFO	( 1 min) operation 99c8304f-c43f5bf8-24dd03e8-4e9bdd9f: running=1     completed=1     pending=0     failed=0     aborted=0     lost=0     total=2     blocked=0    


2025-05-26 17:10:24,958	INFO	( 3 min) operation 99c8304f-c43f5bf8-24dd03e8-4e9bdd9f completed


In [6]:
for record in yt.read_table(result_path):
    print(f"Q: {record['prompt']}")
    print(f"A: {record['text']}")

Q: [{'role': 'user', 'content': "Can a frog fly? If it can't - how can we help it take off?"}]
A:  Frogs are amphibians and do not have the anatomical adaptations necessary for flight. They have evolved to be excellent swimmers and jumpers, using their powerful hind legs to propel themselves through water and leap across land. Their bodies are not aerodynamically suited for flying, and they lack wings.

However, if you're looking for a fun and imaginative way to help a frog "take off" in a playful context, you can create a safe and controlled environment that simulates the sensation of flight. Here are some suggestions:

1. Use a small, soft, and lightweight object, like a helium-filled balloon, as a toy for the frog to "chase." This will create a sense of movement and lightness without posing any risk to the frog.

2. Create a mini obstacle course with elevated platforms or ramps. Encourage the frog to jump from one platform to another, simulating a flight-like experience as it leaps 